# Sentiment Analysis 情绪分析

_Artificial Intelligence Nanodegree Program | Natural Language Processing_

人工智能 Nanodegree 程序 | 自然语言处理

---

With the rise of online social media platforms like Twitter, Facebook and Reddit, and the proliferation of customer reviews on sites like Amazon and Yelp, we now have access, more than ever before, to massive text-based data sets! They can be analyzed in order to determine how large portions of the population feel about certain products, events, etc. This sort of analysis is called _sentiment analysis_. In this notebook you will build an end-to-end sentiment classification system from scratch.

随着在线社交媒体平台（如Twitter，Facebook和Reddit）的兴起以及亚马逊和Yelp等网站上客户评论的激增，我们现在比以往任何时候都可以获得大量基于文本的数据集！ 可以对它们进行分析，以确定人口对某些产品，事件等的大部分感觉。这种分析称为_sentiment analysis_。 在这本笔记本中，您将从头开始构建端到端情绪分类系统。

## Instructions 说明

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

已经为您提供了一些模板代码，您将需要实现其他功能来成功完成此笔记本。 您不需要修改包含的代码超出所要求的内容。 在标题中以“**TODO**”开头的部分表示您需要在其中完成或实现某些部分。 将为每个部分提供说明，实现的具体细节在代码块中标有`＃TODO：...`注释。 请务必仔细阅读说明！

In addition to implementing code, there will be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

除了执行代码之外，还有一些问题可以回答与任务和实现相关的问题。 您将回答问题的每个部分之前都有一个“**Question:**”标题。 仔细阅读每个问题，并通过编辑Markdown单元格在“**Answer:**”标题下方提供答案。

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.
> **Note**：可以使用**Shift + Enter**键盘快捷键执行代码和Markdown单元格。 另外，通过单击它可以编辑单元格（双击Markdown单元格），或者在突出显示时按 **Enter**。

## Step 1: Exploring the data! 第一步：浏览数据

The dataset we are going to use is very popular among researchers in Natural Language Processing, usually referred to as the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). It consists of movie reviews from the website [imdb.com](http://www.imdb.com/), each labeled as either '**pos**itive', if the reviewer enjoyed the film, or '**neg**ative' otherwise.

我们将要使用的数据集在自然语言处理的研究人员中非常受欢迎，通常被称为[IMDb数据集](http://ai.stanford.edu/~amaas/data/sentiment/)。 它包括网站[imdb.com](http://www.imdb.com/) 的电影评论，每个标签为“**pos**itive”，如果评论者喜欢这部电影，或者“**neg**ative”。

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

> Maas，Andrew L.，et al。 [情绪分析学习词汇矢量](http://ai.stanford.edu/~amaas/data/sentiment/)。 在计算语言学协会第49届年会的成果：人类语言技术_ 计算语言学协会，2011年。

We have provided the dataset for you. You can load it in by executing the Python cell below.

我们为您提供了数据集。 您可以通过执行下面的Python单元格来加载它。

In [ ]:
import os
import glob

def read_imdb_data(data_dir='data/imdb-reviews'):
    """Read IMDb movie reviews from given directory.
    
    Directory structure expected:
    - data/
        - train/
            - pos/
            - neg/
        - test/
            - pos/
            - neg/
    
    """

    # Data, labels to be returned in nested dicts matching the dir. structure
    data = {}
    labels = {}

    # Assume 2 sub-directories: train, test
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}

        # Assume 2 sub-directories for sentiment (label): pos, neg
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            # Fetch list of files for this sentiment
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            # Read reviews data and assign labels
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(sentiment)
            
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
    
    # Return data, labels as nested dicts
    return data, labels


data, labels = read_imdb_data()
print("IMDb reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
        len(data['train']['pos']), len(data['train']['neg']),
        len(data['test']['pos']), len(data['test']['neg'])))

Now that the data is loaded in, let's take a quick look at one of the positive reviews:

现在我们加载了数据，让我们快速浏览一下积极的评论：

In [ ]:
print(data['train']['pos'][2])

And one with a negative sentiment:

有负面情绪的人：

In [ ]:
print(data['train']['neg'][2])

We can also make a wordcloud visualization of the reviews.

我们也可以使一个词云可视化的评论。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS

sentiment = 'pos'

# Combine all reviews for the desired sentiment
combined_text = " ".join([review for review in data['train'][sentiment]])

# Initialize wordcloud object
wc = WordCloud(background_color='white', max_words=50,
        # update stopwords to include common words like film and movie
        stopwords = STOPWORDS.update(['br','film','movie']))

# Generate and plot wordcloud
plt.imshow(wc.generate(combined_text))
plt.axis('off')
plt.show()

Try changing the sentiment to `'neg'` and see if you can spot any obvious differences between the wordclouds.

尝试将情绪改为 `'neg'`，看看是否可以发现wordcloud之间的明显差异。

### TODO: Form training and test sets 表格训练和测试集

Now that you've seen what the raw data looks like, combine the positive and negative documents to get one unified training set and one unified test set.

现在您已经看到原始数据的外观，将正面和负面的文档结合起来，得到一个统一的训练集和一个统一的测试集。

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data):
    """Prepare training and test sets from IMDb movie reviews."""
    
    # TODO: Combine positive and negative reviews and labels
    
    # TODO: Shuffle reviews and corresponding labels within training and test sets
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test


data_train, data_test, labels_train, labels_test = prepare_imdb_data(data)
print("IMDb reviews (combined): train = {}, test = {}".format(len(data_train), len(data_test)))

## Step 2. Preprocessing 第2步：预处理

As you might have noticed in the sample reviews, our raw data includes HTML. Therefore there are HTML tags that need to be removed. We also need to remove non-letter characters, normalize uppercase letters by converting them to lowercase, tokenize, remove stop words, and stem the remaining words in each document.

正如你可能已经注意到的样本审查，我们的原始数据包括HTML。 因此，需要删除HTML标签。 我们还需要删除非字母的字符，通过将大写字母转换为小写，标记化，删除停止字，并且阻止每个文档中剩余的单词来规范化大写字母。

### TODO: Convert each review to words 将每个评论转换成单词

As your next task, you should complete the function `review_to_words()` that performs all these steps. For your convenience, in the Python cell below we provide you with all the libraries that you may need in order to accomplish these preprocessing steps. Make sure you can import all of them! (If not, pip install from a terminal and run/import again.)

作为您的下一个任务，您应该完成执行所有这些步骤的函数 `review_to_words()`。 为了方便起见，在下面的Python单元格中，我们为您提供了完成这些预处理步骤所需的所有库。 确保您可以导入所有这些！ （如果没有，从终端安装pip并再次运行/导入。）

In [ ]:
# BeautifulSoup to easily remove HTML tags
from bs4 import BeautifulSoup 

# RegEx for removing non-letter characters
import re

# NLTK library for the remaining steps
import nltk
nltk.download("stopwords")   # download list of stopwords (only once; need not run it again)
from nltk.corpus import stopwords # import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
def review_to_words(review):
    """Convert a raw review string into a sequence of words."""
    
    # TODO: Remove HTML tags and non-letters,
    #       convert to lowercase, tokenize,
    #       remove stopwords and stem

    # Return final list of words
    return words


review_to_words("""This is just a <em>test</em>.<br/><br />
But if it wasn't a test, it would make for a <b>Great</b> movie review!""")

With the function `review_to_words()` fully implemeneted, we can apply it to all reviews in both training and test datasets. This may take a while, so let's build in a mechanism to write to a cache file and retrieve from it later.

使用函数 `review_to_words()` 完全实现，我们可以将其应用于训练和测试数据集中的所有评论。 这可能需要一段时间，所以让我们构建一个写入缓存文件的机制，以便稍后检索。

In [ ]:
import pickle

cache_dir = os.path.join("cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = list(map(review_to_words, data_train))
        words_test = list(map(review_to_words, data_test))
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test


# Preprocess data
words_train, words_test, labels_train, labels_test = preprocess_data(
        data_train, data_test, labels_train, labels_test)

# Take a look at a sample
print("\n--- Raw review ---")
print(data_train[1])
print("\n--- Preprocessed words ---")
print(words_train[1])
print("\n--- Label ---")
print(labels_train[1])

## Step 3: Extracting Bag-of-Words features 提取Bag-of-Words的功能

Now that each document has been preprocessed, we can transform each into a Bag-of-Words feature representation. Note that we need to create this transformation based on the training data alone, as we are not allowed to peek at the testing data at all!
现在每个文档都已被预处理，我们可以将每个文档转换成Bag-of-Words特征表示。 请注意，我们需要根据培训数据单独创建此转换，因为我们不允许窥视测试数据！

The dictionary or _vocabulary_ $V$ (set of words shared by documents in the training set) used here will be the one on which we train our supervised learning algorithm. Any future test data must be transformed in the same way for us to be able to apply the learned model for prediction. Hence, it is important to store the transformation / vocabulary as well.

这里使用的词典或 _vocabulary_ $V$（训练集中的文档共享的单词集合）将是我们训练我们的监督学习算法的词典。 任何未来的测试数据必须以相同的方式进行转换，以使我们能够应用所学习的模型进行预测。 因此，存储转换/词汇也很重要。

> **Note**: The set of words in the training set may not be exactly the same as the test set. What do you do if you encounter a word during testing that you haven't seen before? Unfortunately, we'll have to ignore it, or replace it with a special `<UNK>` token.

> **Note**：训练集中的单词集可能与测试集不完全相同。 如果在测试过程中遇到一个您以前没见过的单词，该怎么办？不幸的是，我们要忽略它，或者用特殊`<UNK>`令牌替换它。

### TODO: Compute Bag-of-Words features 计算Bag-of-Words特征

Implement the `extract_BoW_features()` function, apply it to both training and test datasets, and store the results in `features_train` and `features_test` NumPy arrays, respectively. Choose a reasonable vocabulary size, say $|V| = 5000$, and keep only the top $|V|$ occuring words and discard the rest. This number will also serve as the number of columns in the BoW matrices.

实现`extract_BoW_features()`函数，将其应用于训练和测试数据集，并分别将结果存储在`features_train`和 `features_test`NumPy数组中。 选择合理的词汇大小，如$|V| = 5000$，并且只保留顶部 $|V|$ 发生的单词，并丢弃其余的。 该数字也将用作BoW矩阵中的列数。

> **Hint**: You may find it useful to take advantage of `CountVectorizer` from scikit-learn. Also make sure to pickle your Bag-of-Words transformation so that you can use it in future.

> **Hint**：您可能会发现从scikit学习中利用“CountVectorizer”是有用的。 还要确保腌制你的Bag-of-Words转换，以便将来可以使用它。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # TODO: Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = ?
        features_train = ?

        # TODO: Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = ?
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary


# Extract Bag of Words features for both training and test datasets
features_train, features_test, vocabulary = extract_BoW_features(words_train, words_test)

# Inspect the vocabulary that was computed
print("Vocabulary: {} words".format(len(vocabulary)))

import random
print("Sample words: {}".format(random.sample(list(vocabulary.keys()), 8)))

# Sample
print("\n--- Preprocessed words ---")
print(words_train[5])
print("\n--- Bag-of-Words features ---")
print(features_train[5])
print("\n--- Label ---")
print(labels_train[5])

Let's try to visualize the Bag-of-Words feature vector for one of our training documents.

我们尝试为我们的训练文件中的一个可视化的 Bag-of-Words 特征向量。

In [ ]:
# Plot the BoW feature vector for a training document
plt.plot(features_train[5,:])
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

#### Question: Reflecting on Bag-of-Words feature representation

What is the average sparsity level of BoW vectors in our training set? In other words, on average what percentage of entries in a BoW feature vector are zero?

#### Answer:

...

### Zipf's law 齐普法则

[Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law), named after the famous American linguist George Zipf, is an empirical law stating that given a large collection of documents, the frequency of any word is inversely proportional to its rank in the frequency table. So the most frequent word will occur about twice as often as the second most frequent word, three times as often as the third most frequent word, and so on. In the figure below we plot number of appearances of each word in our training set against its rank.

[齐普法则](https://en.wikipedia.org/wiki/Zipf%27s_law)，以着名的美国语言学家乔治·齐普夫（George Zipf）命名，是一个实证法，规定给予大量文件，任何单词的频率为 与频率表中的等级成反比。 所以最常用的词语将发生的频率是第二个最常用词的两倍，是第三个最常用词的三倍，依此类推。 在下图中，我们将训练集中每个单词的出现次数与其排名相比较。

In [ ]:
# Find number of occurrences for each word in the training set
word_freq = features_train.sum(axis=0)

# Sort it in descending order
sorted_word_freq = np.sort(word_freq)[::-1]

# Plot 
plt.plot(sorted_word_freq)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.xlabel('Rank')
plt.ylabel('Number of occurrences')
plt.show()

#### Question: Zipf's law

What is the total number of occurrences of the most frequent word? What is the the total number of occurrences of the second most frequent word? Do your numbers follow  Zipf's law? If not, why?

#### Answer:

...

### TODO: Normalize feature vectors 归一化特征向量

Bag-of-Words features are intuitive to understand as they are simply word counts. But counts can vary a lot, and potentially throw off learning algorithms later in the pipeline. So, before we proceed further, let's normalize the BoW feature vectors to have unit length.

Bag-of-Words功能是直观的理解，因为它们只是字数。 但是，计数可能会有很大的变化，并且可能会在后期抛弃学习算法。 所以，在我们继续进行之前，让我们将BoW特征向量归一化以具有单位长度。

This makes sure that each document's representation retains the unique mixture of feature components, but prevents documents with large word counts from dominating those with fewer words.

这确保每个文档的表示保留特征组件的独特混合，但是防止具有较大字数的文档以较少的单词为主。

In [ ]:
import sklearn.preprocessing as pr

# TODO: Normalize BoW features in training and test set


## Step 4: Classification using BoW features 使用BoW功能分类

Now that the data has all been properly transformed, we can feed it into a classifier. To get a baseline model, we train a Naive Bayes classifier from scikit-learn (specifically, [`GaussianNB`](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)), and evaluate its accuracy on the test set.

现在数据已经被正确地转换了，我们可以把它分给一个分类器。 为了获得基线模型，我们从scikit学习（具体来说，[`GaussianNB`](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)）训练一个朴素贝叶斯分类器） ，并评估其在测试集上的准确性。

In [ ]:
from sklearn.naive_bayes import GaussianNB

# TODO: Train a Guassian Naive Bayes classifier
clf1 = ?

# Calculate the mean accuracy score on training and test sets
print("[{}] Accuracy: train = {}, test = {}".format(
        clf1.__class__.__name__,
        clf1.score(features_train, labels_train),
        clf1.score(features_test, labels_test)))

Tree-based algorithms often work quite well on Bag-of-Words as their highly discontinuous and sparse nature is nicely matched by the structure of trees. As your next task, you will try to improve on the Naive Bayes classifier's performance by using scikit-learn's Gradient-Boosted Decision Tree classifer.

基于树的算法通常在Bag-of-Words上工作得很好，因为它们的高度不连续和稀疏的性质与树的结构很好地匹配。 作为您的下一个任务，您将尝试通过使用scikit学习的渐变增强决策树类别来改进朴素贝叶斯分类器的性能。

### TODO: Gradient-Boosted Decision Tree classifier 梯度增强决策树分类器

Use [`GradientBoostingClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) from scikit-learn to classify the BoW data. This model has a number of parameters. We use default parameters for some of them and pre-set the rest for you, except one: `n_estimators`. Find a proper value for this hyperparameter, use it to classify the data, and report how much improvement you get over Naive Bayes in terms of accuracy.

使用[`GradientBoostingClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) 从scikit学习将BoW数据分类。 这个模型有很多参数。 我们为其中一些使用默认参数，并为您预留其余参数，除了一个：`n_estimators`。 为此超参数找到一个适当的值，使用它来对数据进行分类，并报告在准确性方面对Naive Bayes有多少改进。

> **Tip**: Use a model selection technique such as cross-validation, grid-search, or an information criterion method, to find an optimal value for the hyperparameter.

> **Tip**：使用模型选择技术，如交叉验证，网格搜索或信息标准方法，以找到超参数的最佳值。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

n_estimators = ?

def classify_gboost(X_train, X_test, y_train, y_test):        
    # Initialize classifier
    clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=1.0, max_depth=1, random_state=0)

    # TODO: Classify the data using GradientBoostingClassifier
    
    # TODO(optional): Perform hyperparameter tuning / model selection
    
    # TODO: Print final training & test accuracy
    
    # Return best classifier model
    return clf


clf2 = classify_gboost(features_train, features_test, labels_train, labels_test)

### TODO: Adverserial testing 对抗测试

Write a short movie review to trick your machine learning model! That is, a movie review with a clear positive or negative sentiment that your model will classify incorrectly.

写一个简短的电影评论来欺骗你的机器学习模型！ 那就是一个电影评论，有一个明确的积极或消极的情绪，你的模型将不正确地分类。

> **Hint**: You might want to take advantage of the biggest weakness of the Bag-of-Words scheme!

> **Hint**：您可能想利用Bag-of-Words计划的最大弱点！

In [ ]:
# TODO: Write a sample review and set its true sentiment
my_review = "..."
true_sentiment = ''  # sentiment must be 'pos' or 'neg'

# TODO: Apply the same preprocessing and vectorizing steps as you did for your training data

# TODO: Then call your classifier to label it


## Step 5: Switching gears - RNNs 切换齿轮 - RNN

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

我们刚刚看到情绪分析的任务可以通过传统的机器学习方法来解决：BoW +一个非线性分类器。 我们现在切换齿轮并使用循环神经网络，特别是LSTM来进行Keras的情感分析。 方便的是，Keras具有内置的[IMDb电影评论数据集](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)，我们可以使用相同的词汇大小。

In [ ]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

In [ ]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

请注意，标签是一个整数（0表示负数，1表示正数），而评价本身存储为整数序列。 这些是已经预先分配给单个单词的单词ID。 要将它们映射回原始单词，可以使用`imdb.get_word_index()`返回的字典。

In [ ]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

与我们的Bag-of-Words方法不同，我们简单地总结了文档中每个单词的计数，该表示基本上保留了整个单词序列（减去标点符号，停用词等）。 这对于RNN起作用至关重要。 但这也意味着现在的功能可以有不同的长度！

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

...


### TODO: Pad sequences Pad序列

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

为了将这些数据提供给您的RNN，所有输入文档的长度必须相同。 我们通过截断更长的评论和填充具有空值（0）的较短评论来限制最大审阅长度为 `max_words` 。 您可以使用Keras中的[`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) 功能轻松完成此任务。 现在，将 `max_words` 设置为500。

In [ ]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test


### TODO: Design an RNN model for sentiment analysis 设计情绪分析的RNN模型

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

在下面的代码单元中构建你的模型体系结构。 我们从Keras导入了一些您可能需要的图层，但可以随意使用您喜欢的任何其他图层/转换。

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

请记住，您的输入是最大长度= max_words的一系列单词（技术上，整数字ID），您的输出是二进制情绪标签（0或1）。

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
model = ?

print(model.summary())

#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model 训练和评估你的模型

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

现在你准备好训练你的模型了。 在Keras世界中，您首先需要_compile_您的模型，方法是在培训时指定要使用的损失函数和优化器，以及您要测量的任何评估指标。 指定适当参数，包括至少一个度量 `'accuracy'`。

In [ ]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics


Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [ ]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = ?
num_epochs = ?

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)

# TODO: Train your model


In [ ]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [ ]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!